# Notebook 05: Evaluación y Resultados

**Prueba Técnica - Modelador Junior**

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [38]:
modelo = joblib.load('../data/results/modelo_final.pkl')
scaler = joblib.load('../data/results/scaler.pkl')
print(f'Modelo: {type(modelo).__name__}')

Modelo: RandomForestClassifier


In [39]:
df = pd.read_csv('../data/processed/dataset_final.csv')
exclude = ['date_requested', 'date_prediction', 'direction_real', 'value_at_request', 'value_at_prediction']
X = df[[c for c in df.columns if c not in exclude]]
y = df['direction_real']

print(f'Dataset cargado: {df.shape}')
print(f'Features (X): {X.shape}')
print(f'Target (y): {y.shape}')

Dataset cargado: (56, 33)
Features (X): (56, 28)
Target (y): (56,)


In [40]:
from sklearn.model_selection import train_test_split

# Split temporal (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
X_test_scaled = scaler.transform(X_test)

print(f'Train set: {X_train.shape}')
print(f'Test set: {X_test.shape}')
print(f'Proporcion: {len(X_train)/len(X):.1%} train / {len(X_test)/len(X):.1%} test')

Train set: (39, 28)
Test set: (17, 28)
Proporcion: 69.6% train / 30.4% test


In [41]:
y_pred = modelo.predict(X_test_scaled)
y_proba = modelo.predict_proba(X_test_scaled)

print("="*70)
print("EVALUACION FINAL DEL MODELO")
print("="*70)
print(f'\nModelo: {type(modelo).__name__}')
print(f'Test set: {len(y_test)} registros')
print("\n" + "-"*70)
print("METRICAS DE CLASIFICACION")
print("-"*70)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f'Accuracy:  {acc:.4f} ({acc*100:.2f}%)')
print(f'Precision: {prec:.4f} ({prec*100:.2f}%)')
print(f'Recall:    {rec:.4f} ({rec*100:.2f}%)')
print(f'F1-Score:  {f1:.4f} ({f1*100:.2f}%)')

print("\n" + "-"*70)
print("MATRIZ DE CONFUSION")
print("-"*70)
cm = confusion_matrix(y_test, y_pred)
print(f'\n              Predicho')
print(f'              Baja  Sube')
print(f'Real  Baja     {cm[0,0]}     {cm[0,1]}')
print(f'      Sube     {cm[1,0]}     {cm[1,1]}')

print("\n" + "-"*70)
print("INTERPRETACION")
print("-"*70)
print(f'[+] Verdaderos Negativos (TN): {cm[0,0]} - Correctamente predijo BAJA')
print(f'[!] Falsos Positivos (FP):     {cm[0,1]} - Predijo SUBE pero fue BAJA')
print(f'[!] Falsos Negativos (FN):     {cm[1,0]} - Predijo BAJA pero fue SUBE')
print(f'[+] Verdaderos Positivos (TP): {cm[1,1]} - Correctamente predijo SUBE')
print(f'\n[OK] Accuracy: {acc*100:.2f}% - {cm[0,0] + cm[1,1]} de {len(y_test)} predicciones correctas')
print(f'[OK] Precision-Recall balanceados: {prec*100:.0f}% cada uno')
print("="*70)

EVALUACION FINAL DEL MODELO

Modelo: RandomForestClassifier
Test set: 17 registros

----------------------------------------------------------------------
METRICAS DE CLASIFICACION
----------------------------------------------------------------------
Accuracy:  0.7647 (76.47%)
Precision: 0.6000 (60.00%)
Recall:    0.6000 (60.00%)
F1-Score:  0.6000 (60.00%)

----------------------------------------------------------------------
MATRIZ DE CONFUSION
----------------------------------------------------------------------

              Predicho
              Baja  Sube
Real  Baja     10     2
      Sube     2     3

----------------------------------------------------------------------
INTERPRETACION
----------------------------------------------------------------------
[+] Verdaderos Negativos (TN): 10 - Correctamente predijo BAJA
[!] Falsos Positivos (FP):     2 - Predijo SUBE pero fue BAJA
[!] Falsos Negativos (FN):     2 - Predijo BAJA pero fue SUBE
[+] Verdaderos Positivos (TP): 3 - C

In [42]:
# Resumen de predicciones
print("="*70)
print("RESUMEN DE PREDICCIONES EN TEST SET")
print("="*70)

print(f"\nTotal de casos en test: {len(y_test)}")
print(f"Predicciones correctas: {(y_pred == y_test.values).sum()}")
print(f"Predicciones incorrectas: {(y_pred != y_test.values).sum()}")

print("\n" + "-"*70)
print("EJEMPLOS DE PREDICCIONES")
print("-"*70)

for i in range(min(5, len(y_pred))):
    print(f"\n[Caso {i+1}]")
    print(f"  Direccion REAL:      {'SUBE' if y_test.values[i] == 1 else 'BAJA'}")
    print(f"  Direccion PREDICHA:  {'SUBE' if y_pred[i] == 1 else 'BAJA'}")
    print(f"  Confianza:           {max(y_proba[i])*100:.1f}%")
    print(f"  Resultado:           {'CORRECTO' if y_pred[i] == y_test.values[i] else 'INCORRECTO'}")

print("\n" + "="*70)
print("[OK] Evaluacion completada")
print("="*70)


RESUMEN DE PREDICCIONES EN TEST SET

Total de casos en test: 17
Predicciones correctas: 13
Predicciones incorrectas: 4

----------------------------------------------------------------------
EJEMPLOS DE PREDICCIONES
----------------------------------------------------------------------

[Caso 1]
  Direccion REAL:      BAJA
  Direccion PREDICHA:  BAJA
  Confianza:           100.0%
  Resultado:           CORRECTO

[Caso 2]
  Direccion REAL:      BAJA
  Direccion PREDICHA:  BAJA
  Confianza:           96.0%
  Resultado:           CORRECTO

[Caso 3]
  Direccion REAL:      SUBE
  Direccion PREDICHA:  BAJA
  Confianza:           82.0%
  Resultado:           INCORRECTO

[Caso 4]
  Direccion REAL:      BAJA
  Direccion PREDICHA:  BAJA
  Confianza:           80.0%
  Resultado:           CORRECTO

[Caso 5]
  Direccion REAL:      BAJA
  Direccion PREDICHA:  BAJA
  Confianza:           84.0%
  Resultado:           CORRECTO

[OK] Evaluacion completada


## Conclusiones (3-4 bullets)

**Resumen:**
- El modelo [nombre] logró un F1-Score de X.XX%, demostrando capacidad para predecir la dirección de precios
- Las features más importantes fueron [top 3], indicando que el consenso entre modelos es clave
- El dataset balanceado permitió entrenar sin necesidad de técnicas de balanceo

**Limitaciones:**
- Dataset pequeño (~17 registros) limita la generalización del modelo
- Limitado a un solo commodity (Granular), no generalizable a otros productos
- Diferencia fijo de 4 semanas, no adaptable a otros períodos

**Próximos pasos:**
- Recolectar más datos históricos para mejorar robustez
- Probar con múltiples commodities para modelo más general
- Implementar ensemble más sofisticado (stacking, boosting)
- Agregar features temporales y de mercado externo